In [2]:
pretty = True
highres = True

%matplotlib inline
if highres:
    %config InlineBackend.figure_format = 'retina'
else:
    %config InlineBackend.figure_format = 'png'

#rcParams["figure.dpi"]=300

import sys

pypsapath = "C:/dev/py/PyPSA/"

if sys.path[0] != pypsapath:
    sys.path.insert(0,pypsapath)

%load_ext autoreload
%autoreload 2

In [3]:
import pypsa
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt

from IPython.display import Markdown, display
printm = lambda s: display(Markdown(s))

In [4]:
csv_folder_name = pypsapath + "examples/scigrid-de/scigrid-with-load-gen-trafos/"

network = pypsa.Network(import_name=csv_folder_name)

Importing PyPSA from older version of PyPSA than current version 0.13.2.
Please read the release notes at https://pypsa.org/doc/release_notes.html
carefully to prepare your network for import.

INFO:pypsa.io:Imported network  has buses, generators, lines, loads, storage_units, transformers


# Overview

In [5]:
printm(f"There are {len(network.buses)} buses, {len(network.generators)} generators and {len(network.loads)} loads.")
printm(f"There are {len(network.storage_units)} storage units, of which {sum(network.storage_units.carrier == 'Pumped Hydro')} pumped hydro")
printm(f"There are {len(network.lines)} lines at voltage levels {network.buses.v_nom.unique()} kV")
printm(f"There are {len(network.transformers)} transformers")

There are 585 buses, 1423 generators and 489 loads.

There are 38 storage units, of which 38 pumped hydro

There are 852 lines at voltage levels [220. 380.] kV

There are 96 transformers

In [6]:
carrier_counts = network.generators.carrier.value_counts()
printm(f"**Number of generators per type**")
for carrier in ["Solar", "Wind Onshore", "Wind Offshore"]:
    print(f"{carrier}: {carrier_counts[carrier]}")

**Number of generators per type**

Solar: 489
Wind Onshore: 488
Wind Offshore: 5


# Loads

In [7]:
double_load_buses = sum(network.loads.bus.value_counts()>1)
printm(f"There are {double_load_buses} buses that house more than one load.")
if double_load_buses == 0:
    printm("No bus houses more than one load. Realistically, this is highly unlikely, which means that multiple loads (i.e. two cities in the same region) are modeled as a single loads. (This is known as _load aggregation_.)")

There are 0 buses that house more than one load.

No bus houses more than one load. Realistically, this is highly unlikely, which means that multiple loads (i.e. two cities in the same region) are modeled as a single loads. (This is known as _load aggregation_.)

------------------
# Bus pairs

Each generator or load connects to exactly one bus, and a single bus can house multiple generators and loads.

Let's check whether each generator and load can be linked to a bus in the dataset, and then count the number of buses with no generators or loads (which probably have transformers or transmisison lines connected to them, otherwise they would not be included in the dataset).

First, we let's take a look at the bus names.

In [8]:
bus_names = list(network.buses.index)
print(bus_names)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '

Buses are labeled 1 through 495 (although some numbers are skipped, like '243'). Some numbers are used again, with the `_220kV` suffix. These double numbers probably indicate a substation that transforms between 380kV and 220kV, with lines, generators or loads connected to both voltages. The substation therefore houses a transformer, and it is modeled as two distinct nodes. Let's check this assumption:

## Transformers between bus pairs

In [9]:
bus_names_220_suffix = [n for n in bus_names if n[-6:]=="_220kV"]
printm(f"{len(bus_names_220_suffix)} bus names end in `_220kV`. Compare this to the number of transformers: {len(network.transformers)}")

suffix_removed = [n[:-6] for n in bus_names_220_suffix]
printm(f"After removing the `_220kV` suffix, {sum((n in bus_names) for n in suffix_removed)} buses have a second bus with the same name.")

pairs = list(zip(suffix_removed, bus_names_220_suffix))
number_of_transformer_connected_pairs = 0

for threethirty_name, twotwenty_name in pairs:
    number_of_transformer_connected_pairs += ((network.transformers.bus0 == threethirty_name) & (network.transformers.bus0 == threethirty_name)).any()

printm(f"Of these 380-220 bus pairs, {number_of_transformer_connected_pairs} are connected by a transformer.")

96 bus names end in `_220kV`. Compare this to the number of transformers: 96

After removing the `_220kV` suffix, 96 buses have a second bus with the same name.

Of these 380-220 bus pairs, 96 are connected by a transformer.

## Generators at bus pairs

In [10]:
generator_buses = network.generators.bus.unique()

printm(f"Of the {len(pairs)} 380-220 bus pairs, {sum((n in generator_buses) and (n220 in generator_buses) for n, n220 in pairs)} have generators connected to both buses.")

any_non_aggregated_buses = False
for n,df in network.generators.groupby('bus'):
    if (df.carrier.value_counts() > 1).any():
        printm(f"Bus {n} houses multiple generators of the same type")
        any_non_aggregated_buses = True

if not any_non_aggregated_buses:
    printm("No bus houses more than one generator of the same type. Realistically, this is highly unlikely, which means that multiple generators (i.e. two solar parks in the same region) are modeled as a single generator.")

Of the 96 380-220 bus pairs, 0 have generators connected to both buses.

No bus houses more than one generator of the same type. Realistically, this is highly unlikely, which means that multiple generators (i.e. two solar parks in the same region) are modeled as a single generator.

## Loads at bus pairs

In [11]:
load_buses = network.loads.bus.unique()

printm(f"Of the {len(pairs)} 380-220 bus pairs, {sum((n in load_buses) and (n220 in load_buses) for n, n220 in pairs)} have loads connected to both buses.")

Of the 96 380-220 bus pairs, 0 have loads connected to both buses.

## Loads _or_ generators at bus pairs

In [12]:
printm(f"{sum((n in load_buses or n in generator_buses) and (n220 in load_buses or n220 in generator_buses) for n, n220 in pairs)} pairs do not house generators and loads on the same bus.")
printm(f"{sum((n220 in load_buses or n220 in generator_buses) for n, n220 in pairs)} pairs have a load or generator connected to the `_220kV` bus.")

0 pairs do not house generators and loads on the same bus.

96 pairs have a load or generator connected to the `_220kV` bus.

## Conclusion

In the case of the SciGRID dataset for Germany, there are 96 pairs of buses with the same name. For each pair, one of them operates at 220kV and the other at 380kV. These pairs correspond exactly to the 96 transformers in the dataset.

At each pair, all loads and generators are connected to the 220kV bus.

This means that in this dataset, **some loads and generators are connected to a 380kV bus _via a 220-380 transformer_**, but never the other way around. 

If we consider those 380-220 bus pairs as a single node, we find that **in this particular model**:
* **Every node houses a single load.**
* **Every node houses a single solar generator.** Almost every node houses a single onshore wind generator. Some nodes house a single offshore wind generator.

## Unpaired buses
Some buses have no '220' double. This does not mean that these buses are 380kV buses:

In [13]:
unpaired_buses = network.buses[[(n not in bus_names_220_suffix) for n in bus_names]]
unpaired_bus_voltages = unpaired_buses.v_nom.value_counts()

printm(f"Of the remaining {len(unpaired_buses)} buses, there are")
for voltage, num in unpaired_bus_voltages.iteritems():
    printm(f"{num} buses at {voltage}kV")

Of the remaining 489 buses, there are

288 buses at 380.0kV

201 buses at 220.0kV

These unpaired 220kV buses also house generators and loads:

In [14]:
unpaired_220_buses = unpaired_buses[unpaired_buses.v_nom < 300]

printm(f"The {len(unpaired_220_buses)} unpaired 220kV buses house a total of {sum(b in unpaired_220_buses.index for b in network.generators.bus)} generators and {sum(b in unpaired_220_buses.index for b in network.loads.bus)} loads.")


The 201 unpaired 220kV buses house a total of 600 generators and 201 loads.

# Generators per bus

In [78]:
total_capacity = [(carrier,np.sum(df.p_nom.values)) for carrier, df in network.generators.groupby('carrier')]
sorted_carriers = sorted(total_capacity, key=lambda p: p[1], reverse=True)
[(n,"{:.2g}".format(c/1000)) for n,c in sorted_carriers]

[('Wind Onshore', '37'),
 ('Solar', '37'),
 ('Hard Coal', '25'),
 ('Gas', '24'),
 ('Brown Coal', '21'),
 ('Nuclear', '12'),
 ('Run of River', '4'),
 ('Other', '3'),
 ('Wind Offshore', '3'),
 ('Oil', '2.7'),
 ('Waste', '1.6'),
 ('Storage Hydro', '1.4'),
 ('Multiple', '0.15'),
 ('Geothermal', '0.032')]

In [15]:
renewable_carriers = ["Solar", "Wind Onshore", "Wind Offshore"]
generator_buses_renewable = []

for n, df in network.generators.groupby('bus'):
    if sum((t in renewable_carriers) for t in df.carrier):
        generator_buses_renewable.append(n)

printm(f"There are {len(generator_buses_renewable)} buses housing renewable generators.")
if(len(generator_buses_renewable) == len(generator_buses)):
    printm(f"This means that every substation in the network houses a renewable generator.")


generator_buses_purely_gray = [n for n in generator_buses if n not in generator_buses_renewable]
printm(f"There are {len(network.buses)-len(generator_buses_renewable)} buses that do not house a renewable generator, of which {len(generator_buses_purely_gray)} house a non-renewable generator.")

There are 489 buses housing renewable generators.

This means that every substation in the network houses a renewable generator.

There are 96 buses that do not house a renewable generator, of which 0 house a non-renewable generator.

----

# Parallel lines

I noticed a number of parallel lines in the network, i.e. two or more lines with the same start and end point. Some of these are a result of combining the two voltage buses (e.g. a line `123 <-> 234` and a line `123_220kV <-> 234_220kV`), but some are even parallel in the original network.

In [49]:
old_lines = list(map(tuple, network.lines[["bus0", "bus1"]].values))

printm(f"The original network contains {len(old_lines)} lines, of which only {len(set(old_lines))} are unique.")

The original network contains 852 lines, of which only 705 are unique.

How many are unique, _after_ combining 220 and 380 buses?

In [38]:
# We re-index the nodes, ie 123 and 123_220kV get the same index.
def node_index(bus_name):
    if bus_name in suffix_removed:
        return node_index(bus_name + "_220kV")
    return sorted(list(generator_buses)).index(bus_name)

In [51]:
new_lines = [(node_index(a), node_index(b)) for a, b in old_lines]

In [85]:
printm(f"The original network contains {len(old_lines)} lines, of which only {len(set(map(tuple,(map(sorted,new_lines)))))} are unique, after combining 220kV and 380kV.")

The original network contains 852 lines, of which only 695 are unique, after combining 220kV and 380kV.

We can map the old indices to the unique bus pair that they connect:

In [57]:
old_line_indices = {l: [i for i, l_old in enumerate(old_lines) if l_old == l] for l in old_lines}
scigrid_line_indices = [[network.lines.index.values[i] for i in indices] for indices in old_line_indices.values()]

old_line_indices

{('1', '2_220kV'): [0, 416],
 ('3', '4'): [1, 420],
 ('5_220kV', '6'): [2, 425],
 ('7', '5'): [3],
 ('8', '9'): [4],
 ('10_220kV', '11'): [5],
 ('11', '12_220kV'): [6],
 ('10_220kV', '12_220kV'): [7],
 ('13_220kV', '14'): [8, 667],
 ('13', '15'): [9, 13],
 ('16', '5'): [10, 433],
 ('17', '18'): [11],
 ('17', '12'): [12],
 ('14', '15_220kV'): [14],
 ('13_220kV', '19'): [15],
 ('20', '21'): [16, 368],
 ('20', '22'): [17, 367],
 ('20_220kV', '23'): [18],
 ('23', '24_220kV'): [19],
 ('25', '26'): [20],
 ('25', '22'): [21],
 ('23', '27'): [22],
 ('28', '23'): [23],
 ('8', '21'): [24, 668],
 ('9', '29'): [25, 438],
 ('30', '25'): [26],
 ('31', '32'): [27],
 ('32', '33'): [28],
 ('34', '35'): [29, 365],
 ('35', '36'): [30, 359],
 ('2_220kV', '6'): [31],
 ('37', '10'): [32],
 ('10', '38'): [33],
 ('37', '38'): [34],
 ('39', '40'): [35],
 ('39', '41'): [36],
 ('42', '41'): [37],
 ('18', '42'): [38],
 ('10_220kV', '43'): [39, 40],
 ('44', '45'): [41],
 ('44', '46_220kV'): [42],
 ('46', '12'): [4

In [20]:
new_nodes = sorted(list(generator_buses))


In [21]:
m = len(new_lines)
{l: [i for i in range(m) if new_lines[i]==l] for l in set(new_lines)}

{(458, 459): [66],
 (99, 332): [76],
 (381, 392): [839],
 (156, 157): [316],
 (383, 367): [690],
 (479, 474): [91],
 (459, 253): [506, 507],
 (219, 112): [419],
 (133, 463): [570],
 (18, 19): [116],
 (86, 360): [283],
 (221, 483): [432],
 (179, 129): [465],
 (336, 337): [608, 609],
 (474, 378): [710],
 (436, 437): [49, 655],
 (10, 7): [110],
 (87, 23): [212],
 (445, 417): [785],
 (16, 475): [113],
 (6, 475): [107],
 (200, 198): [455],
 (146, 437): [304],
 (73, 396): [748],
 (219, 445): [31],
 (251, 349): [634, 635],
 (20, 58): [160],
 (317, 316): [572],
 (258, 433): [817],
 (137, 142): [642],
 (243, 307): [558],
 (73, 406): [764],
 (105, 106): [825],
 (381, 385): [846],
 (263, 261): [516],
 (10, 385): [722],
 (259, 465): [484, 550],
 (55, 56): [157],
 (85, 397): [752],
 (406, 409): [849],
 (228, 233): [449],
 (196, 198): [375, 376],
 (220, 331): [1, 420],
 (129, 140): [301],
 (471, 163): [87],
 (456, 444): [3],
 (20, 399): [753],
 (463, 464): [71, 72],
 (48, 50): [146],
 (238, 424): [7

In [22]:
network.lines.iloc[[608, 609],:]

,bus0,bus1,c_nfkm,cables,frequency,from_relation,i_th_max_a,length,length_m,num_parallel,...,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
name,,,,,,,,,,,,,,,,,,,,,
612,403_220kV,404_220kV,NaN,3.0,NaN,3916290.0,NaN,10.791,10791.0,1.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
613,403,404,NaN,3.0,NaN,3916291.0,NaN,9.656,9656.0,0.5,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [87]:
network.lines.iloc[[561,567,568],:]

,bus0,bus1,c_nfkm,cables,frequency,from_relation,i_th_max_a,length,length_m,num_parallel,...,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
name,,,,,,,,,,,,,,,,,,,,,
565,382_220kV,308_220kV,11.5,3.0,50.0,3866867.0,1.3,2.735,2735.0,1.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
571,382,308,NaN,3.0,NaN,3866874.0,NaN,2.265,2265.0,0.5,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
572,382,308,13.7,3.0,NaN,3866875.0,2.6,2.958,2958.0,1.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
